In [12]:
# Load data
import pandas as pd
pew = pd.read_csv('../dataset/pew_dataset/metadata.csv')
pew['imgPath'] = pew['imgPath'].str.replace('imgs', '../dataset/pew_dataset/pew_imgs')
statista = pd.read_csv('../dataset/statista_dataset/metadata.csv')
statista['imgPath'] = statista['imgPath'].str.replace('out/two_col/imgs', '../dataset/statista_dataset/statista_imgs')
columns = ['title', 'caption', 'imgPath']

pew_df = pew[columns]
statista_df = statista[columns]
combined_df = pd.concat([pew_df, statista_df], ignore_index=True)
combined_df.insert(0, 'id', combined_df.reset_index().index + 1)
combined_df

,id,title,caption,imgPath
0,1,"Foreign-born population in the United States, ...",The foreign-born population residing in the U....,../dataset/pew_dataset/pew_imgs/1.png
1,2,"English proficiency among U.S. immigrants, 198...","Since 1980, the share of immigrants who are pr...",../dataset/pew_dataset/pew_imgs/2.png
2,3,"Languages spoken among U.S. immigrants, 2018","Among the nation’s immigrants, Spanish is by f...",../dataset/pew_dataset/pew_imgs/3.png
3,4,"Hispanic population in the U.S., 2000-2017",There were nearly 60 million Latinos in the Un...,../dataset/pew_dataset/pew_imgs/4.png
4,5,Weekly broadcast audience for top 20 NPR-affil...,The top 20 NPR-affiliated public radio station...,../dataset/pew_dataset/pew_imgs/5.png
...,...,...,...,...
29349,29350,\r\n Distribution of cy...,This statistic presents the distribution of c...,../dataset/statista_dataset/statista_imgs/2786...
29350,29351,\r\n Total number of dw...,This statistic displays the total number of d...,../dataset/statista_dataset/statista_imgs/2786...
29351,29352,\r\n Results in the Eur...,This statistic shows the political parties an...,../dataset/statista_dataset/statista_imgs/2786...
29352,29353,\r\n Average annual exp...,This statistic shows the average annual expen...,../dataset/statista_dataset/statista_imgs/2786...


In [13]:
# Add caption lengths
combined_df['title_length'] = combined_df['title'].apply(len)
combined_df['caption_length'] = combined_df['caption'].apply(len)

combined_df

,id,title,caption,imgPath,title_length,caption_length
0,1,"Foreign-born population in the United States, ...",The foreign-born population residing in the U....,../dataset/pew_dataset/pew_imgs/1.png,55,392
1,2,"English proficiency among U.S. immigrants, 198...","Since 1980, the share of immigrants who are pr...",../dataset/pew_dataset/pew_imgs/2.png,52,480
2,3,"Languages spoken among U.S. immigrants, 2018","Among the nation’s immigrants, Spanish is by f...",../dataset/pew_dataset/pew_imgs/3.png,44,404
3,4,"Hispanic population in the U.S., 2000-2017",There were nearly 60 million Latinos in the Un...,../dataset/pew_dataset/pew_imgs/4.png,42,434
4,5,Weekly broadcast audience for top 20 NPR-affil...,The top 20 NPR-affiliated public radio station...,../dataset/pew_dataset/pew_imgs/5.png,66,282
...,...,...,...,...,...,...
29349,29350,\r\n Distribution of cy...,This statistic presents the distribution of c...,../dataset/statista_dataset/statista_imgs/2786...,113,228
29350,29351,\r\n Total number of dw...,This statistic displays the total number of d...,../dataset/statista_dataset/statista_imgs/2786...,176,295
29351,29352,\r\n Results in the Eur...,This statistic shows the political parties an...,../dataset/statista_dataset/statista_imgs/2786...,146,189
29352,29353,\r\n Average annual exp...,This statistic shows the average annual expen...,../dataset/statista_dataset/statista_imgs/2786...,176,256


In [17]:
title_sorted=combined_df.sort_values(by='title_length', ascending=False)
title_sorted

,id,title,caption,imgPath,title_length,caption_length
60,61,Confidence in public health organizations like...,This survey finds that this is indeed the case...,../dataset/pew_dataset/pew_imgs/61.png,757,616
669,670,A Snapshot of What Americans Know About Scienc...,Most Americans (86%) correctly identify the Ea...,../dataset/pew_dataset/pew_imgs/670.png,620,537
695,696,"How Mixed-Race, Mestizo, 'uulatto' Hispanics R...",The use of these labels to describe mixed-race...,../dataset/pew_dataset/pew_imgs/696.png,567,635
916,917,"Write- Ins for ""Some Other Race"" Among Hispani...",According to a new Census report released last...,../dataset/pew_dataset/pew_imgs/917.png,520,397
62,63,Roughly three-in-ten who say social media have...,When asked to elaborate on the main reason why...,../dataset/pew_dataset/pew_imgs/63.png,507,575
...,...,...,...,...,...,...
902,903,Incomes,But the American rich still make considerably ...,../dataset/pew_dataset/pew_imgs/903.png,7,194
1467,1468,AGURE,According to the 2002 National Survey of Latin...,../dataset/pew_dataset/pew_imgs/1468.png,6,1210
1466,1467,CHART,Data from the 2002 National Survey of Latinos ...,../dataset/pew_dataset/pew_imgs/1467.png,6,466
1469,1470,CHART,This analysis divides Latinos into three group...,../dataset/pew_dataset/pew_imgs/1470.png,6,411


In [15]:
caption_sorted=combined_df.sort_values(by='caption_length', ascending=False)
caption_sorted

,id,title,caption,imgPath,title_length,caption_length
11346,11347,\r\n Population of Pola...,"Throughout the nineteenth century, what we kn...",../dataset/statista_dataset/statista_imgs/9861...,154,5110
8186,8187,\r\n Number of assassin...,"Throughout U.S. history, there have been coun...",../dataset/statista_dataset/statista_imgs/6701...,148,4509
7487,7488,\r\n Population of Gree...,"Prior to 1829, the area of modern day Greece ...",../dataset/statista_dataset/statista_imgs/6002...,154,4488
6540,6541,\r\n Population of Fran...,"During the eighteenth century, it is estimate...",../dataset/statista_dataset/statista_imgs/5055...,154,4327
4069,4070,\r\n Reported number of...,Of the first eighteen presidents of the Unite...,../dataset/statista_dataset/statista_imgs/2584...,212,4071
...,...,...,...,...,...,...
25836,25837,\r\n Change in survey r...,The statistic shows the influence of televise...,../dataset/statista_dataset/statista_imgs/2435...,127,88
7050,7051,\r\n The highest mounta...,The Statisitk shows the highest mountains in ...,../dataset/statista_dataset/statista_imgs/5565...,145,87
10807,10808,\r\n Market forecast of...,This statistic gives a market forecast of the...,../dataset/statista_dataset/statista_imgs/9322...,129,75
11011,11012,\r\n Structure of the g...,The statistic shows the structure of the glob...,../dataset/statista_dataset/statista_imgs/9526...,95,74


In [18]:
from transformers import CLIPTokenizer, CLIPModel

# Load the CLIP model and tokenizer
model_name = "openai/clip-vit-large-patch14"
tokenizer = CLIPTokenizer.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)

# Function to tokenize the title using CLIP and get token length
def get_clip_token_length(text):
    tokens = tokenizer(text, return_tensors="pt")
    return tokens.input_ids.size(1)

# Apply the function to each title
title_sorted['token_length'] = title_sorted['title'].apply(get_clip_token_length)

# Select only the required columns
title_sorted = title_sorted[['id', 'title', 'token_length']]

filtered_title = title_sorted[title_sorted['token_length'] > 77]
filtered_title



Token indices sequence length is longer than the specified maximum sequence length for this model (141 > 77). Running this sequence through the model will result in indexing errors


,id,title,token_length
60,61,Confidence in public health organizations like...,141
669,670,A Snapshot of What Americans Know About Scienc...,128
695,696,"How Mixed-Race, Mestizo, 'uulatto' Hispanics R...",141
916,917,"Write- Ins for ""Some Other Race"" Among Hispani...",134
62,63,Roughly three-in-ten who say social media have...,109
799,800,"The Web IQ"" of American Internet Users % of fi...",96
660,661,Foreign- Born Hispanics More Likely to Be Cath...,84
63,64,25% of Americans who say social media have a p...,81
93,94,Majority of parents say their child 11 or youn...,89
414,415,The largest number of single-mother households...,82


In [19]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")

# Since Sentence Transformers use Hugging Face Transformers models, we can access the tokenizer directly
tokenizer = model.tokenizer

# Function to tokenize the caption and get token length
def get_token_length(text):
    # Tokenize the text and count the number of input IDs, which represent tokens
    tokens = tokenizer(text, return_tensors="pt")
    return tokens['input_ids'].shape[1]  # Get the number of tokens

# Apply the function to each caption in your DataFrame
caption_sorted['caption_token_length'] = caption_sorted['caption'].apply(get_token_length)

# Select only the required columns to view the results
caption_sorted = caption_sorted[['id', 'title', 'caption', 'caption_token_length']]

caption_sorted=caption_sorted.sort_values(by='caption_token_length', ascending=False)
caption_sorted


/home/mleshashi/thesis-sharma/sraEnv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 